In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split,KFold

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)


In [2]:
train=pd.read_csv("./data/train.csv")#(891,12)
test=pd.read_csv("./data/test.csv")#(418,11)
train['is_train']=1
test['is_train']=0
data=pd.concat([train,test])

In [334]:
data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,is_train,Cabins,familyNum,has_family,CabinA,CabinB,CabinC,CabinD,CabinE,CabinF,CabinG,CabinT,CabinZ,EmbarkedC,EmbarkedQ,EmbarkedS,TitleMaster,TitleMiss,TitleMr,TitleMrs,TitleRare,Sexfemale,Sexmale,female1,female2,female3,male1,male2,male3
0,0.0,3,22.0,1,0,7.2500,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1
1,1.0,1,38.0,1,0,71.2833,1,1,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0
2,1.0,3,26.0,0,0,7.9250,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0
3,1.0,1,35.0,1,0,53.1000,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0
4,0.0,3,35.0,0,0,8.0500,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1


# 数据预处理
## 数据预处理

In [3]:
data['Fare']=data['Fare'].fillna(data['Fare'].median())# 有一个空值，中位数填充

data['Embarked']=data['Embarked'].fillna("S")# 就2个缺失值，填充了众数

# Cabin只有204个非空值
data['Cabins']=data['Cabin'].apply(lambda x:2 if len(str(x))>4 else(1 if pd.notna(x) else 0) )
data['Cabin']=data['Cabin'].str[:1]
data['Cabin']=data['Cabin'].fillna("Z")

# ticket比较复杂，先算了
# train['Ticket'].apply(lambda x:x.split(" ")[0]).value_counts()

data['Title']=data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt','Don','Col','Dr','Rev', 'Major', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
data['Title'] = data['Title'].replace('Mlle', 'Miss')
data['Title'] = data['Title'].replace('Ms', 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')

data['familyNum']=data['SibSp']+data['Parch']
data['has_family']=data['familyNum'].apply(lambda x:1 if x>0 else 0)

data=data.drop(['Name','Ticket','PassengerId'],axis=1)

In [55]:
# ohe = OneHotEncoder(handle_unknown='ignore')

# ohe.fit(train[['Cabin','Embarked','Title','Sex']])#训练规则
# feature_names=ohe.get_feature_names(['Cabin','Embarked','Title','Sex'])#获取编码后的特征名
# data_train_onehot=pd.DataFrame(ohe.transform(train[['Cabin','Embarked','Title','Sex']]).toarray(),columns=feature_names)#应用规则在训练集上
 
# data_new_onehot=pd.DataFrame(ohe.transform(test[['Cabin','Embarked','Title','Sex']]).toarray(),columns=feature_names)#应用规则在预测集上

In [ ]:
for col in ['Cabin','Embarked','Title','Sex']:
    df=pd.get_dummies(data[col],sparse=True)
    for c in df.columns:
        df.rename(columns={c:col+c},inplace=True)
    data=pd.concat([data.drop([col],axis=1),df],axis=1)

# df=pd.get_dummies(data['SexPclass'],sparse=True)
# data=pd.concat([data.drop(['SexPclass'],axis=1),df],axis=1)

# data=data.drop(['CabinA','CabinD','CabinF','CabinG','CabinT','Sexmale'],axis=1)

## 预测测试集是否生存，用来填充缺失值

In [6]:
X_train=np.array(data[data['is_train']==1].drop(['Survived','is_train'],axis=1))
y_train=np.array(data[data['is_train']==1]['Survived'])
X_test=np.array(data[data['is_train']==0].drop(['Survived','is_train'],axis=1))

In [7]:
from lightgbm import LGBMClassifier

best_model=None
folds=5
avg_score=0
best_score=0
result=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=LGBMClassifier(learning_rate=0.07,max_depth=32,n_estimators=70)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score+=score
    result+=lgb.predict(X_test)
avg_score/=folds
result/=folds

print("avg score= ",avg_score)

0 th fold： 0.8044692737430168
1 th fold： 0.7696629213483146
2 th fold： 0.8033707865168539
3 th fold： 0.9101123595505618
4 th fold： 0.8539325842696629
avg score=  0.8283095850856821


In [350]:
data[data['is_train']==1].drop(['Survived','is_train'],axis=1).columns# CabinA,CabinD,CabinF,CabinG,CabinT,Sexmale

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabins', 'familyNum',
       'has_family', 'CabinB', 'CabinC', 'CabinE', 'CabinZ', 'EmbarkedC',
       'EmbarkedQ', 'EmbarkedS', 'TitleMaster', 'TitleMiss', 'TitleMr',
       'TitleMrs', 'TitleRare', 'Sexfemale', 'female1', 'female2', 'female3',
       'male1', 'male2', 'male3'],
      dtype='object')

In [351]:
lgb.feature_importances_

array([ 47, 711,  31,  30, 676,  22,  57,   1,   6,  18,   4,   5,  25,
        10,  59,  15,  21,  26,  22,   6,  27,  17,  16,   8,   2,   9,
        19])

## age缺失值填充
### 用lgb预测

In [8]:
# X_test_pred=pd.read_csv("./result_20220522_ss.txt").iloc[:,1]# 历史最好成绩survival预测成绩
X_test_pred=best_model.predict(X_test)
data.iloc[891:,0]=X_test_pred
data=data.reset_index(drop=True)

In [9]:
data_age=data[data['Age'].notnull()]
data_noAge=data[data['Age'].isnull()]

X_train_age=np.array(data_age.drop(['is_train','Age'],axis=1))
y_train_age=np.array(data_age['Age'])
X_test_age=np.array(data_noAge.drop(['is_train','Age'],axis=1))

In [10]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

best_model=None
folds=5
avg_score=0
best_score=0

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train_age)):
    lgb=LGBMRegressor(learning_rate=0.5,max_depth=5,n_estimators=7)
    lgb.fit(X_train_age[train_index],y_train_age[train_index])
    
    
    pred=lgb.predict(X_train_age[train_index])
    score0=mean_squared_error(pred,y_train_age[train_index])
    print(i,'th fold 0：',score0)
    
    pred=lgb.predict(X_train_age[test_index])
    score=mean_squared_error(pred,y_train_age[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score+=score
avg_score/=folds

print("avg score= ",avg_score)

0 th fold 0： 92.07688785077193
0 th fold： 116.11721678928332
1 th fold 0： 92.07857399680114
1 th fold： 125.98550188876987
2 th fold 0： 93.60144946658201
2 th fold： 123.24232168543234
3 th fold 0： 93.33004877316607
3 th fold： 115.24514689872464
4 th fold 0： 96.41907057424133
4 th fold： 95.65952433752226
avg score=  115.24994231994648


In [11]:
age_pred=best_model.predict(X_test_age)
data.iloc[data[data['Age'].isnull()].index,2]=age_pred

# 模型训练

In [12]:
X_train=np.array(data[data['is_train']==1].drop(['Survived','is_train'],axis=1))
y_train=np.array(data[data['is_train']==1]['Survived'])
X_test=np.array(data[data['is_train']==0].drop(['Survived','is_train'],axis=1))

## lgb

In [130]:
from lightgbm import LGBMClassifier

best_model=None
folds=5
best_score=0
avg_score_lgb=0
result_lgb=np.zeros(418)
train_result_lgb=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=LGBMClassifier(learning_rate=0.06,max_depth=12,n_estimators=80)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
        
    avg_score_lgb+=score
    result_lgb+=lgb.predict_proba(X_test)[:,1]
    train_result_lgb[test_index]=lgb.predict_proba(X_train[test_index])[:,1]
    
avg_score_lgb/=folds
result_lgb/=folds

print("avg score= ",avg_score_lgb)

0 th fold： 0.8435754189944135
1 th fold： 0.7865168539325843
2 th fold： 0.8146067415730337
3 th fold： 0.9044943820224719
4 th fold： 0.8707865168539326
avg score=  0.8439959826752872


## xgb

In [148]:
from xgboost import XGBClassifier

best_model=None
folds=5
best_score=0
avg_score_xgb=0
result_xgb=np.zeros(418)
train_result_xgb=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=XGBClassifier(learning_rate=0.016,max_depth=16,n_estimators=73)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_xgb+=score
    result_xgb+=lgb.predict_proba(X_test)[:,1]
    train_result_xgb[test_index]=lgb.predict_proba(X_train[test_index])[:,1]
    
avg_score_xgb/=folds
result_xgb/=5

print("avg score= ",avg_score_xgb)

[21:04:08] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0 th fold： 0.8547486033519553
[21:04:08] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1 th fold： 0.8202247191011236
[21:04:08] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
2 th fold： 0.8314606741573034
[21:04:08] WARNING: D:\bld\xgboost-split_16

In [143]:
best_line=pd.read_csv("./results/result_20220522_ss_gbt.txt").iloc[:,1]
np.where(np.round(result_xgb)!=best_line)

(array([  4,  28,  32,  39,  68,  73,  86,  87,  90, 125, 138, 144, 148,
        159, 165, 181, 192, 197, 283, 331, 344, 383, 404, 405], dtype=int64),)

## catboost

In [151]:
from catboost import CatBoostClassifier
best_model=None
folds=5
best_score=0
avg_score_cat=0
result_cat=np.zeros(418)
train_result_cat=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=CatBoostClassifier(learning_rate=0.13,n_estimators=82)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_cat+=score
    result_cat+=lgb.predict_proba(X_test)[:,1]
    train_result_cat[test_index]=lgb.predict_proba(X_train[test_index])[:,1]
    
avg_score_cat/=folds
result_cat/=5

print("avg score= ",avg_score_cat)

0:	learn: 0.6085408	total: 1.31ms	remaining: 106ms
1:	learn: 0.5650476	total: 2.05ms	remaining: 81.9ms
2:	learn: 0.5234278	total: 3.26ms	remaining: 85.8ms
3:	learn: 0.4820338	total: 4.4ms	remaining: 85.9ms
4:	learn: 0.4559431	total: 5.69ms	remaining: 87.7ms
5:	learn: 0.4392616	total: 6.89ms	remaining: 87.3ms
6:	learn: 0.4226631	total: 8.04ms	remaining: 86.1ms
7:	learn: 0.4102533	total: 9.11ms	remaining: 84.3ms
8:	learn: 0.4015329	total: 10.2ms	remaining: 82.9ms
9:	learn: 0.3970154	total: 11.1ms	remaining: 79.7ms
10:	learn: 0.3892624	total: 12.3ms	remaining: 79.2ms
11:	learn: 0.3803759	total: 13.4ms	remaining: 78.2ms
12:	learn: 0.3784468	total: 14ms	remaining: 74.3ms
13:	learn: 0.3766813	total: 14.9ms	remaining: 72.2ms
14:	learn: 0.3715771	total: 16ms	remaining: 71.2ms
15:	learn: 0.3653191	total: 17ms	remaining: 70.3ms
16:	learn: 0.3593422	total: 18ms	remaining: 68.9ms
17:	learn: 0.3525755	total: 19.3ms	remaining: 68.6ms
18:	learn: 0.3500315	total: 20.4ms	remaining: 67.7ms
19:	learn: 0.

0:	learn: 0.6292978	total: 13.9ms	remaining: 1.12s
1:	learn: 0.5736805	total: 15.3ms	remaining: 612ms
2:	learn: 0.5294578	total: 16.4ms	remaining: 432ms
3:	learn: 0.4908416	total: 17.5ms	remaining: 341ms
4:	learn: 0.4704738	total: 18.6ms	remaining: 286ms
5:	learn: 0.4626350	total: 19ms	remaining: 240ms
6:	learn: 0.4451549	total: 20ms	remaining: 214ms
7:	learn: 0.4302467	total: 21ms	remaining: 194ms
8:	learn: 0.4210643	total: 22ms	remaining: 178ms
9:	learn: 0.4170755	total: 22.6ms	remaining: 163ms
10:	learn: 0.4069832	total: 23.6ms	remaining: 153ms
11:	learn: 0.3996352	total: 24.6ms	remaining: 144ms
12:	learn: 0.3941316	total: 25.9ms	remaining: 137ms
13:	learn: 0.3892232	total: 27.1ms	remaining: 132ms
14:	learn: 0.3866593	total: 28.2ms	remaining: 126ms
15:	learn: 0.3820987	total: 29.2ms	remaining: 121ms
16:	learn: 0.3799100	total: 30.3ms	remaining: 116ms
17:	learn: 0.3748635	total: 31.3ms	remaining: 111ms
18:	learn: 0.3697674	total: 32.3ms	remaining: 107ms
19:	learn: 0.3669177	total: 33

## gbt

In [168]:
from sklearn.ensemble import GradientBoostingClassifier
best_model=None
folds=5
best_score=0
avg_score_gbt=0
result_gbt=np.zeros(418)
train_result_gbt=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=GradientBoostingClassifier(learning_rate=0.1,n_estimators=220)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_gbt+=score
    result_gbt+=lgb.predict_proba(X_test)[:,1]
    train_result_gbt[test_index]=lgb.predict_proba(X_train[test_index])[:,1]
    
avg_score_gbt/=folds
result_gbt/=folds

print("avg score= ",avg_score_gbt)

0 th fold： 0.8324022346368715
1 th fold： 0.7921348314606742
2 th fold： 0.8089887640449438
3 th fold： 0.9213483146067416
4 th fold： 0.8539325842696629
avg score=  0.8417613458037788


## RF

In [177]:
from sklearn.ensemble import RandomForestClassifier
best_model=None
folds=5
best_score=0
avg_score_rf=0
result_rf=np.zeros(418)
train_result_rf=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=RandomForestClassifier(n_estimators=150,max_depth=5,min_samples_split=5,min_samples_leaf=3)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_rf+=score
    result_rf+=lgb.predict_proba(X_test)[:,1]
    train_result_rf[test_index]=lgb.predict_proba(X_train[test_index])[:,1]
    
avg_score_rf/=folds
result_rf/=folds

print("avg score= ",avg_score_rf)

0 th fold： 0.8435754189944135
1 th fold： 0.8202247191011236
2 th fold： 0.8033707865168539
3 th fold： 0.8820224719101124
4 th fold： 0.8202247191011236
avg score=  0.8338836231247253


## lr

In [178]:
from sklearn.linear_model import LogisticRegression
best_model=None
folds=5
best_score=0
avg_score_lr=0
result_lr=np.zeros(418)
train_result_lr=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=LogisticRegression()
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_lr+=score
    result_lr+=lgb.predict_proba(X_test)[:,1]
    train_result_lr[test_index]=lgb.predict_proba(X_train[test_index])[:,1]
    
avg_score_lr/=folds
result_lr/=folds

print("avg score= ",avg_score_lr)

0 th fold： 0.8324022346368715
1 th fold： 0.7808988764044944
2 th fold： 0.797752808988764
3 th fold： 0.8932584269662921
4 th fold： 0.8146067415730337
avg score=  0.8237838177138912


## SVC

In [202]:
from sklearn.svm import LinearSVC

best_model=None
folds=5
best_score=0
avg_score_svc=0
result_svc=np.zeros(418)
train_result_svc=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=LinearSVC(C=0.12,tol=1e-4,max_iter=1000)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_svc+=score
    result_svc+=lgb.predict(X_test)
    train_result_svc[test_index]=lgb.predict(X_train[test_index])
    
avg_score_svc/=folds
result_svc/=folds

print("avg score= ",avg_score_svc)

0 th fold： 0.8268156424581006
1 th fold： 0.8146067415730337
2 th fold： 0.8033707865168539
3 th fold： 0.8764044943820225
4 th fold： 0.8089887640449438
avg score=  0.8260372857949909


## knn

In [209]:
from sklearn.neighbors import KNeighborsClassifier

best_model=None
folds=5
best_score=0
avg_score_knn=0
result_knn=np.zeros(418)
train_result_knn=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=KNeighborsClassifier(n_neighbors=5,p=1)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_knn+=score
    result_knn+=lgb.predict_proba(X_test)[:,1]
    train_result_knn[test_index]=lgb.predict_proba(X_train[test_index])[:,1]
    
avg_score_knn/=folds
result_knn/=folds

print("avg score= ",avg_score_knn)

0 th fold： 0.7486033519553073
1 th fold： 0.8089887640449438
2 th fold： 0.7415730337078652
3 th fold： 0.8258426966292135
4 th fold： 0.8033707865168539
avg score=  0.7856757265708368


## perceptron

In [216]:
from sklearn.linear_model import Perceptron

best_model=None
folds=5
best_score=0
avg_score_pt=0
result_pt=np.zeros(418)
train_result_pt=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=Perceptron(penalty='l2',alpha=2e-6,max_iter=10,validation_fraction=0.1,n_iter_no_change=10,warm_start=True)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_pt+=score
    result_pt+=lgb.predict(X_test)
    train_result_pt[test_index]=lgb.predict(X_train[test_index])
    
avg_score_pt/=folds
result_pt/=folds

print("avg score= ",avg_score_pt)

0 th fold： 0.7262569832402235
1 th fold： 0.797752808988764
2 th fold： 0.7471910112359551
3 th fold： 0.8314606741573034
4 th fold： 0.7640449438202247
avg score=  0.7733412842884941


## SGD

In [227]:
from sklearn.linear_model import SGDClassifier

best_model=None
folds=5
best_score=0
avg_score_sgd=0
result_sgd=np.zeros(418)
train_result_sgd=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=SGDClassifier(loss='log',alpha=1e-3,max_iter=20,validation_fraction=0.1,n_iter_no_change=10,warm_start=True)
#     lgb=SGDClassifier(penalty='l2',alpha=1e-5,max_iter=10,validation_fraction=0.1,n_iter_no_change=10,warm_start=True)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_sgd+=score
    result_sgd+=lgb.predict(X_test)
    train_result_sgd[test_index]=lgb.predict(X_train[test_index])
#     result_sgd+=lgb.predict_proba(X_test)[:,1]
#     train_result_sgd[test_index]=lgb.predict_proba(X_train[test_index])[:,1]
    
avg_score_sgd/=folds
result_sgd/=folds

print("avg score= ",avg_score_sgd)

0 th fold： 0.7653631284916201
1 th fold： 0.8202247191011236
2 th fold： 0.7528089887640449
3 th fold： 0.8202247191011236
4 th fold： 0.7247191011235955
avg score=  0.7766681313163015


## gp

In [228]:
from sklearn.gaussian_process import GaussianProcessClassifier

best_model=None
folds=5
best_score=0
avg_score_gp=0
result_gp=np.zeros(418)
train_result_gp=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=GaussianProcessClassifier()
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_gp+=score
    result_gp+=lgb.predict_proba(X_test)[:,1]
    train_result_gp[test_index]=lgb.predict_proba(X_train[test_index])[:,1]
    
avg_score_gp/=folds
result_gp/=folds

print("avg score= ",avg_score_gp)

0 th fold： 0.8156424581005587
1 th fold： 0.7078651685393258
2 th fold： 0.6910112359550562
3 th fold： 0.7696629213483146
4 th fold： 0.7134831460674157
avg score=  0.739532986002134


##  bayes

In [229]:
from sklearn.naive_bayes import BernoulliNB

best_model=None
folds=5
best_score=0
avg_score_by=0
result_by=np.zeros(418)
train_result_by=np.zeros(891)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=BernoulliNB(alpha=20)
    lgb.fit(X_train[train_index],y_train[train_index])
    score=lgb.score(X_train[test_index],y_train[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_by+=score
    result_by+=lgb.predict_proba(X_test)[:,1]
    train_result_by[test_index]=lgb.predict_proba(X_train[test_index])[:,1]
    
avg_score_by/=folds
result_by/=folds

print("avg score= ",avg_score_by)

0 th fold： 0.8044692737430168
1 th fold： 0.7921348314606742
2 th fold： 0.7528089887640449
3 th fold： 0.8595505617977528
4 th fold： 0.7696629213483146
avg score=  0.7957253154227606


# 模型融合
## 简单平均

In [230]:
result1=(result_lgb+result_xgb+result_cat+result_gbt+result_rf+result_lr+result_svc+result_knn+result_pt+result_sgd+result_gp+result_by)/12

In [198]:
result1_round=np.round(result1)
pd.DataFrame(data=np.c_[list(range(892,1310)),result1_round],columns=["PassengerId","Survived"],dtype=np.int64).to_csv('result_20220522_avg_bestSuvivalAge.txt',index=False,header=True)# 得分：0.73444

In [234]:
baseline=pd.read_csv("./results/result_20220519_avg.txt").iloc[:,1]
np.where(np.round(result1)!=baseline)

(array([ 19,  28,  34,  72,  75, 118, 142, 158, 225, 242, 252, 280, 316,
        323, 347, 390, 412], dtype=int64),)

## 加权平均

In [236]:
result2=(avg_score_lgb*result_lgb\
        +avg_score_xgb*result_xgb\
        +avg_score_cat*result_cat\
        +avg_score_gbt*result_gbt\
        +avg_score_rf*result_rf\
        +avg_score_lr*result_lr\
        +avg_score_svc*result_svc\
        +avg_score_knn*result_knn\
        +avg_score_pt*result_pt\
        +avg_score_sgd*result_sgd\
        +avg_score_gp*result_gp\
        +avg_score_by*result_by)/(avg_score_lgb+avg_score_xgb+avg_score_cat+avg_score_gbt+avg_score_rf+avg_score_lr+avg_score_svc\
                                 +avg_score_knn+avg_score_pt+avg_score_sgd+avg_score_gp+avg_score_by)

In [237]:
np.where(np.round(result2)!=np.round(result1))# 简单平均和加权平均结果一样

(array([316], dtype=int64),)

## stacking

In [238]:
train_stacking=np.c_[train_result_lgb,train_result_xgb,train_result_cat,train_result_gbt,train_result_rf,train_result_lr,train_result_svc,\
      train_result_knn,train_result_pt,train_result_sgd,train_result_gp,train_result_by]
y_train_stacking=y_train.copy()
test_stacking=np.c_[result_lgb,result_xgb,result_cat,result_gbt,result_rf,result_lr,result_svc,\
      result_knn,result_pt,result_sgd,result_gp,result_by]

In [333]:
# 带上原数据的stacking
train_stacking=np.c_[X_train,train_result_lgb,train_result_xgb,train_result_cat,train_result_gbt,train_result_rf,train_result_lr,train_result_svc,\
      train_result_knn,train_result_pt,train_result_sgd,train_result_gp,train_result_by]
y_train_stacking=y_train.copy()
test_stacking=np.c_[X_test,result_lgb,result_xgb,result_cat,result_gbt,result_rf,result_lr,result_svc,\
      result_knn,result_pt,result_sgd,result_gp,result_by]

### rf

In [239]:
best_model=None
folds=5
best_score=0
avg_score_rf_s=0
result_rf_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=RandomForestClassifier(n_estimators=70,max_depth=8,min_samples_split=4,min_samples_leaf=3)
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_rf_s+=score
    result_rf_s+=lgb.predict(test_stacking)
    
avg_score_rf_s/=folds
result_rf_s/=folds

print("avg score= ",avg_score_rf_s)

0 th fold： 0.8268156424581006
1 th fold： 0.8033707865168539
2 th fold： 0.8146067415730337
3 th fold： 0.9157303370786517
4 th fold： 0.848314606741573
avg score=  0.8417676228736426


In [143]:
np.where(np.round(result_rf_s)!=np.round(result1))

(array([  4,   6,  19,  21,  32,  33,  36,  37,  86, 125, 138, 158, 159,
        165, 169, 181, 192, 197, 199, 225, 242, 249, 252, 268, 280, 283,
        293, 323, 347, 359, 367, 382, 383, 412], dtype=int64),)

In [167]:
pd.DataFrame(data=np.c_[list(range(892,1310)),result_lr_s],columns=["PassengerId","Survived"],dtype=np.int64).to_csv('result_20220520_rf_s.txt',index=False,header=True)# 得分：0.73444

### lr

In [240]:
best_model=None
folds=5
best_score=0
avg_score_lr_s=0
result_lr_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=LogisticRegression()
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_lr_s+=score
    result_lr_s+=lgb.predict(test_stacking)
    
avg_score_lr_s/=folds
result_lr_s/=folds

print("avg score= ",avg_score_lr_s)

0 th fold： 0.8379888268156425
1 th fold： 0.7808988764044944
2 th fold： 0.8202247191011236
3 th fold： 0.9101123595505618
4 th fold： 0.8426966292134831
avg score=  0.8383842822170611


In [145]:
np.where(np.round(result_lr_s)!=np.round(result1))

(array([  6,  19,  28,  36,  37,  41, 138, 158, 165, 169, 225, 242, 249,
        252, 268, 280, 283, 323, 347, 412], dtype=int64),)

### lgb

In [241]:
best_model=None
folds=5
best_score=0
avg_score_lgb_s=0
result_lgb_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=LGBMClassifier(learning_rate=0.2,max_depth=10,n_estimators=100)
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_lgb_s+=score
    result_lgb_s+=lgb.predict(test_stacking)
    
avg_score_lgb_s/=folds
result_lgb_s/=folds

print("avg score= ",avg_score_lgb_s)

0 th fold： 0.8268156424581006
1 th fold： 0.7865168539325843
2 th fold： 0.8258426966292135
3 th fold： 0.898876404494382
4 th fold： 0.848314606741573
avg score=  0.8372732408511707


In [457]:
pd.DataFrame(data=np.c_[list(range(892,1310)),np.round(result_lgb_s)],columns=["PassengerId","Survived"],dtype=np.int64).to_csv('result_20220522_sso_lgb.txt',index=False,header=True)# 得分：0.73444

In [456]:
np.where(np.round(result_lgb_s)!=np.round(result1))

(array([  4,   6,  18,  19,  32,  33,  36,  37,  64,  72,  86,  87,  90,
        125, 138, 153, 158, 159, 165, 169, 181, 192, 197, 199, 200, 202,
        225, 242, 249, 252, 268, 273, 280, 283, 293, 344, 345, 347, 359,
        367, 382, 383, 412], dtype=int64),)

### cat

In [242]:
best_model=None
folds=5
best_score=0
avg_score_cat_s=0
result_cat_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=CatBoostClassifier(learning_rate=0.14,n_estimators=70)
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_cat_s+=score
    result_cat_s+=lgb.predict(test_stacking)
    
avg_score_cat_s/=folds
result_cat_s/=folds

print("avg score= ",avg_score_cat_s)

0:	learn: 0.6261120	total: 3.82ms	remaining: 263ms
1:	learn: 0.5794638	total: 4.95ms	remaining: 168ms
2:	learn: 0.5458153	total: 6.28ms	remaining: 140ms
3:	learn: 0.5142316	total: 7.24ms	remaining: 119ms
4:	learn: 0.4885095	total: 8.29ms	remaining: 108ms
5:	learn: 0.4669942	total: 9.24ms	remaining: 98.5ms
6:	learn: 0.4495646	total: 10.1ms	remaining: 90.6ms
7:	learn: 0.4350862	total: 10.9ms	remaining: 84.3ms
8:	learn: 0.4218912	total: 11.7ms	remaining: 79.1ms
9:	learn: 0.4110226	total: 12.3ms	remaining: 73.6ms
10:	learn: 0.4000356	total: 13.1ms	remaining: 70ms
11:	learn: 0.3926617	total: 13.9ms	remaining: 67ms
12:	learn: 0.3856142	total: 14.7ms	remaining: 64.3ms
13:	learn: 0.3794251	total: 15.5ms	remaining: 61.9ms
14:	learn: 0.3732273	total: 16.3ms	remaining: 59.8ms
15:	learn: 0.3682686	total: 17.1ms	remaining: 57.8ms
16:	learn: 0.3632503	total: 17.9ms	remaining: 55.9ms
17:	learn: 0.3599870	total: 18.7ms	remaining: 54.1ms
18:	learn: 0.3564826	total: 19.5ms	remaining: 52.5ms
19:	learn: 0

0:	learn: 0.6283389	total: 1.12ms	remaining: 77.2ms
1:	learn: 0.5795725	total: 2.36ms	remaining: 80.3ms
2:	learn: 0.5426227	total: 3.56ms	remaining: 79.5ms
3:	learn: 0.5107787	total: 4.66ms	remaining: 76.8ms
4:	learn: 0.4850111	total: 5.67ms	remaining: 73.7ms
5:	learn: 0.4636749	total: 6.72ms	remaining: 71.7ms
6:	learn: 0.4468429	total: 7.75ms	remaining: 69.7ms
7:	learn: 0.4320438	total: 8.79ms	remaining: 68.2ms
8:	learn: 0.4194646	total: 9.85ms	remaining: 66.7ms
9:	learn: 0.4099536	total: 10.9ms	remaining: 65.3ms
10:	learn: 0.4004107	total: 12ms	remaining: 64.4ms
11:	learn: 0.3921594	total: 13.1ms	remaining: 63.5ms
12:	learn: 0.3844673	total: 14.2ms	remaining: 62.3ms
13:	learn: 0.3766045	total: 15.3ms	remaining: 61.2ms
14:	learn: 0.3711235	total: 16.4ms	remaining: 60ms
15:	learn: 0.3658858	total: 17.4ms	remaining: 58.8ms
16:	learn: 0.3616976	total: 18.5ms	remaining: 57.5ms
17:	learn: 0.3573479	total: 19.5ms	remaining: 56.3ms
18:	learn: 0.3550255	total: 20.5ms	remaining: 55ms
19:	learn

In [43]:
np.where(np.round(result_cat_s)!=np.round(result1))

(array([ 21,  28,  32,  33,  36,  37,  41,  73,  75,  87,  98, 125, 138,
        144, 148, 153, 158, 165, 169, 197, 199, 202, 242, 249, 252, 268,
        283, 293, 316, 323, 359, 367, 382, 390], dtype=int64),)

### svc

In [243]:
best_model=None
folds=5
best_score=0
avg_score_svc_s=0
result_svc_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=LinearSVC(C=0.008,tol=1e-4,max_iter=100)
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_svc_s+=score
    result_svc_s+=lgb.predict(test_stacking)
    
avg_score_svc_s/=folds
result_svc_s/=folds

print("avg score= ",avg_score_svc_s)

0 th fold： 0.8324022346368715
1 th fold： 0.8089887640449438
2 th fold： 0.8033707865168539
3 th fold： 0.8764044943820225
4 th fold： 0.8258426966292135
avg score=  0.8294017952419811


In [380]:
np.where(np.round(result_svc_s)!=np.round(result1))

(array([  6,  19,  21,  33,  34,  36,  37,  73,  75, 118, 138, 158, 169,
        268, 280, 293, 390, 412], dtype=int64),)

### knn

In [244]:
best_model=None
folds=5
best_score=0
avg_score_knn_s=0
result_knn_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=KNeighborsClassifier(n_neighbors=27,p=1)
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_knn_s+=score
    result_knn_s+=lgb.predict(test_stacking)
    
avg_score_knn_s/=folds
result_knn_s/=folds

print("avg score= ",avg_score_knn_s)

0 th fold： 0.8268156424581006
1 th fold： 0.8033707865168539
2 th fold： 0.8258426966292135
3 th fold： 0.9213483146067416
4 th fold： 0.8314606741573034
avg score=  0.8417676228736426


In [120]:
np.where(np.round(result_knn_s)!=np.round(result1))

(array([  6,  19,  32,  33,  36,  98, 125, 159, 181, 197, 242, 249, 252,
        268, 280, 293, 359, 367], dtype=int64),)

### gbt

In [245]:
best_model=None
folds=5
best_score=0
avg_score_gbt_s=0
result_gbt_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=GradientBoostingClassifier(learning_rate=0.06,n_estimators=73)
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_gbt_s+=score
    result_gbt_s+=lgb.predict(test_stacking)
    
avg_score_gbt_s/=folds
result_gbt_s/=folds

print("avg score= ",avg_score_gbt_s)

0 th fold： 0.8603351955307262
1 th fold： 0.8314606741573034
2 th fold： 0.8146067415730337
3 th fold： 0.9101123595505618
4 th fold： 0.848314606741573
avg score=  0.8529659155106396


In [255]:
pd.DataFrame(data=np.c_[list(range(892,1310)),np.round(result_gbt_s)],columns=["PassengerId","Survived"],dtype=np.int64).to_csv('result_20220523_s_gbt.txt',index=False,header=True)# 得分：0.73444

In [301]:
np.where(np.round(result_gbt_s)!=np.round(result1))

(array([  4,   6,  19,  21,  32,  33,  36,  37,  72,  86,  90, 125, 153,
        158, 159, 165, 169, 181, 192, 197, 199, 225, 242, 245, 249, 252,
        268, 280, 283, 293, 323, 344, 347, 359, 367, 382, 383, 412],
       dtype=int64),)

In [303]:
np.where(np.round(result_rf_s)!=np.round(result_gbt_s))

(array([ 72,  90, 138, 153, 245, 344], dtype=int64),)

In [304]:
np.where(np.round(result_lgb_s)!=np.round(result_gbt_s))

(array([ 18,  21,  64,  87, 138, 200, 202, 245, 273, 323, 345], dtype=int64),)

### xgb

In [246]:
best_model=None
folds=5
best_score=0
avg_score_xgb_s=0
result_xgb_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=XGBClassifier(learning_rate=0.07,max_depth=4,n_estimators=61)
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_xgb_s+=score
    result_xgb_s+=lgb.predict(test_stacking)
    
avg_score_xgb_s/=folds
result_xgb_s/=folds

print("avg score= ",avg_score_xgb_s)

[21:21:53] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0 th fold： 0.8603351955307262
[21:21:53] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1 th fold： 0.8146067415730337
[21:21:53] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
2 th fold： 0.8202247191011236
[21:21:53] WARNING: D:\bld\xgboost-split_16

### perceptron

In [247]:
best_model=None
folds=5
best_score=0
avg_score_pt_s=0
result_pt_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=Perceptron(penalty='l2',alpha=1e-3,max_iter=10,validation_fraction=0.1,n_iter_no_change=20,warm_start=True)
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_pt_s+=score
    result_pt_s+=lgb.predict(test_stacking)
    
avg_score_pt_s/=folds
result_pt_s/=folds

print("avg score= ",avg_score_pt_s)

0 th fold： 0.8156424581005587
1 th fold： 0.7303370786516854
2 th fold： 0.7865168539325843
3 th fold： 0.6685393258426966
4 th fold： 0.797752808988764
avg score=  0.7597577051032578


### sgd

In [249]:
best_model=None
folds=5
best_score=0
avg_score_sgd_s=0
result_sgd_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=SGDClassifier(alpha=1e-4,max_iter=10,validation_fraction=0.1,n_iter_no_change=10,warm_start=True)
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_sgd_s+=score
    result_sgd_s+=lgb.predict(test_stacking)
    
avg_score_sgd_s/=folds
result_sgd_s/=folds

print("avg score= ",avg_score_sgd_s)

0 th fold： 0.8212290502793296
1 th fold： 0.7752808988764045
2 th fold： 0.7584269662921348
3 th fold： 0.9101123595505618
4 th fold： 0.797752808988764
avg score=  0.812560416797439


### bayes

In [250]:
best_model=None
folds=5
best_score=0
avg_score_by_s=0
result_by_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=BernoulliNB(alpha=1)
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_by_s+=score
    result_by_s+=lgb.predict(test_stacking)
    
avg_score_by_s/=folds
result_by_s/=folds

print("avg score= ",avg_score_by_s)

0 th fold： 0.7932960893854749
1 th fold： 0.8370786516853933
2 th fold： 0.7640449438202247
3 th fold： 0.8595505617977528
4 th fold： 0.797752808988764
avg score=  0.8103446111355218


### gp

In [251]:
best_model=None
folds=5
best_score=0
avg_score_gp_s=0
result_gp_s=np.zeros(418)

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(train_stacking)):
    lgb=GaussianProcessClassifier()
    lgb.fit(train_stacking[train_index],y_train_stacking[train_index])
    score=lgb.score(train_stacking[test_index],y_train_stacking[test_index])
    print(i,'th fold：',score)
    if best_score<score:
        best_score=score
        best_model=lgb    
    avg_score_gp_s+=score
    result_gp_s+=lgb.predict(test_stacking)
    
avg_score_gp_s/=folds
result_gp_s/=folds

print("avg score= ",avg_score_gp_s)

0 th fold： 0.8324022346368715
1 th fold： 0.797752808988764
2 th fold： 0.8314606741573034
3 th fold： 0.9157303370786517
4 th fold： 0.8539325842696629
avg score=  0.8462557278262507


### 平均

In [252]:
result_feature_1=(result_lgb_s+result_xgb_s+result_cat_s\
          +result_gbt_s+result_rf_s+result_lr_s\
         +result_svc_s+result_knn_s+result_pt_s\
          +result_sgd_s+result_gp_s+result_by_s)/12

result_feature_2=(avg_score_lgb_s*result_lgb_s\
        +avg_score_xgb_s*result_xgb_s\
        +avg_score_cat_s*result_cat_s\
        +avg_score_gbt_s*result_gbt_s\
        +avg_score_rf_s*result_rf_s\
        +avg_score_lr_s*result_lr_s\
        +avg_score_svc_s*result_svc_s\
        +avg_score_knn_s*result_knn_s\
        +avg_score_pt_s*result_pt_s\
        +avg_score_sgd_s*result_sgd_s\
        +avg_score_gp_s*result_gp_s\
        +avg_score_by_s*result_by_s)/(avg_score_lgb_s+avg_score_xgb_s+avg_score_cat_s+avg_score_gbt_s+avg_score_rf_s+avg_score_lr_s\
                                      +avg_score_svc_s+avg_score_knn_s+avg_score_pt_s+avg_score_sgd_s+avg_score_gp_s+avg_score_by_s)

In [253]:
np.where(np.round(result_feature_1)!=np.round(result_feature_2))

(array([], dtype=int64),)

In [254]:
pd.DataFrame(data=np.c_[list(range(892,1310)),np.round(result_feature_1)],columns=["PassengerId","Survived"],dtype=np.int64).to_csv('result_20220523_s_avg.txt',index=False,header=True)# 得分：0.73444

### 取前k个最好的

In [ ]:
gbt 0.850   0.842
lgb 0.8485  0.848
cat 0.846   0.847
rf 0.845    0.841
xgb 0.840   0.846
gp 0.836    0.765
lr 0.835    0.825
svc 0.835   0.835
knn 0.833   0.813
bayes 0.808 0.806
sgd 0.802   0.819
perceptron 0.792  0.793

In [471]:
result_1=(np.round(result_lgb_s)+np.round(result_xgb_s)+np.round(result_cat_s)\
          +np.round(result_gbt_s)+np.round(result_rf_s)+np.round(result_lr_s)\
          +np.round(result_svc_s)+np.round(result_knn_s)+np.round(result_pt_s)\
          +np.round(result_sgd_s)+np.round(result_gp_s)+np.round(result_by_s))/12

In [459]:
np.where(np.round(resulto11)!=np.round(resulto22))

(array([359], dtype=int64),)

In [328]:
result111[np.where(np.round(result111)!=np.round(result_9))]

array([0.58333333, 0.58333333, 0.58333333])

In [329]:
result_9[np.where(np.round(result111)!=np.round(result_9))]

array([0.44444444, 0.44444444, 0.44444444])

In [461]:
np.where(np.round(result_lgb_s)!=np.round(resulto22))

(array([ 18,  32,  64,  87,  90, 153, 181, 192, 197, 200, 202, 273, 344,
        345, 367, 382, 383], dtype=int64),)

In [323]:
np.where(np.round(result_gbt_s)!=np.round(result_rf_s))

(array([ 72,  90, 138, 153, 245, 344], dtype=int64),)

In [324]:
np.where(np.round(result_gbt_s)!=np.round(result_xgb_s))

(array([ 34,  36, 245, 268, 273, 316, 323, 331, 412], dtype=int64),)

In [313]:
result111[np.where(np.round(result11)!=np.round(result111))]

array([0.5       , 0.5       , 0.5       , 0.48333333, 0.5       ])

In [297]:
np.where(np.round(result11)!=np.round(result1))

(array([  6,  19,  21,  33,  36,  37, 138, 158, 159, 165, 169, 197, 225,
        242, 249, 252, 268, 280, 283, 293, 323, 347, 367, 382, 412],
       dtype=int64),)

In [298]:
result11[np.where(np.round(result11)!=np.round(result1))]

array([0.18333333, 0.33333333, 0.55      , 0.36666667, 0.3       ,
       0.21666667, 0.43333333, 0.85      , 0.46666667, 0.28333333,
       0.18333333, 0.5       , 0.28333333, 0.21666667, 0.23333333,
       0.2       , 0.3       , 0.21666667, 0.28333333, 0.4       ,
       0.51666667, 0.33333333, 0.46666667, 0.5       , 0.4       ])

In [299]:
result1[np.where(np.round(result11)!=np.round(result1))]

array([0.52840008, 0.52437674, 0.45679954, 0.56448001, 0.55540819,
       0.55393837, 0.56096024, 0.43054273, 0.67248507, 0.59836365,
       0.55539291, 0.65666669, 0.58359148, 0.54862118, 0.58336977,
       0.52958313, 0.55540819, 0.52283612, 0.62816882, 0.52198703,
       0.36962838, 0.54255157, 0.68228422, 0.71778027, 0.52814656])

In [137]:
result11[np.where(np.round(result11)!=np.round(result1))]

array([0.18333333, 0.25      , 0.31666667, 0.25      , 0.25      ,
       0.43333333, 0.86666667, 0.5       , 0.25      , 0.23333333,
       0.45      , 0.25      , 0.18333333, 0.2       , 0.18333333,
       0.25      , 0.2       , 0.25      , 0.4       , 0.36666667,
       0.36666667])

In [138]:
result1[np.where(np.round(result11)!=np.round(result1))]

array([0.52840008, 0.52437674, 0.56448001, 0.55540819, 0.55393837,
       0.56096024, 0.43054273, 0.67248507, 0.59836365, 0.55539291,
       0.65666669, 0.58359148, 0.54862118, 0.58336977, 0.52958313,
       0.55540819, 0.52283612, 0.62816882, 0.52198703, 0.54255157,
       0.52814656])

In [135]:
np.where(np.round(result22)!=np.round(result1))

(array([  6,  19,  33,  36,  37, 138, 158, 159, 165, 169, 197, 225, 242,
        249, 252, 268, 280, 283, 293, 347, 412], dtype=int64),)

In [472]:
pd.DataFrame(data=np.c_[list(range(892,1310)),np.round(result_1)],columns=["PassengerId","Survived"],dtype=np.int64).to_csv('result_20220522_featurePclassSex_avg.txt',index=False,header=True)# 得分：0.73444